# Chapter 12: Closures and Higher-Order Functions

In Python, functions are **first-class objects** -- they can be assigned to variables, passed as arguments,
and returned from other functions. This enables powerful patterns like closures and higher-order functions,
which form the foundation of functional programming in Python.

## Key Concepts
- **First-class functions**: Functions are objects that can be stored, passed, and returned
- **Higher-order functions**: Functions that accept or return other functions
- **Closures**: Functions that capture variables from their enclosing scope
- **Lambda expressions**: Anonymous, inline functions for simple operations

## Section 1: First-Class Functions

In Python, functions are objects just like integers, strings, or lists. You can assign them to variables,
store them in data structures, and pass them around freely.

In [ ]:
# Functions are objects -- they can be assigned to variables
def shout(text: str) -> str:
    return text.upper() + "!"

def whisper(text: str) -> str:
    return text.lower() + "..."

# Assign functions to variables (no parentheses = no call)
yell = shout
quiet = whisper

print(yell("hello"))      # HELLO!
print(quiet("HELLO"))     # hello...

# Functions can be stored in data structures
formatters: dict[str, callable] = {
    "loud": shout,
    "soft": whisper,
}

for style, formatter in formatters.items():
    print(f"{style}: {formatter('good morning')}")

In [ ]:
# Passing functions as arguments
from typing import Callable

def apply_to_list(func: Callable[[int], int], values: list[int]) -> list[int]:
    """Apply a function to each element in a list."""
    return [func(v) for v in values]

def square(x: int) -> int:
    return x * x

def negate(x: int) -> int:
    return -x

numbers = [1, 2, 3, 4, 5]
print(f"Original:  {numbers}")
print(f"Squared:   {apply_to_list(square, numbers)}")
print(f"Negated:   {apply_to_list(negate, numbers)}")

## Section 2: Higher-Order Functions

A higher-order function either takes a function as an argument, returns a function, or both.
This is the key building block for functional composition.

In [ ]:
# Higher-order function: accepts a function and applies it twice
from typing import Callable

def apply_twice(f: Callable[[int], int], x: int) -> int:
    """Apply function f to x twice: f(f(x))."""
    return f(f(x))

# apply_twice with add-3: (10 + 3) + 3 = 16
result1 = apply_twice(lambda x: x + 3, 10)
print(f"apply_twice(x + 3, 10) = {result1}")

# apply_twice with double: (3 * 2) * 2 = 12
result2 = apply_twice(lambda x: x * 2, 3)
print(f"apply_twice(x * 2, 3) = {result2}")

# Higher-order function: returns a function
def make_adder(n: int) -> Callable[[int], int]:
    """Return a function that adds n to its argument."""
    def adder(x: int) -> int:
        return x + n
    return adder

add_5 = make_adder(5)
add_10 = make_adder(10)

print(f"\nadd_5(3)  = {add_5(3)}")
print(f"add_10(3) = {add_10(3)}")

## Section 3: Closures

A **closure** is a function that remembers and has access to variables from its enclosing scope,
even after the outer function has finished executing. The inner function "closes over" those variables.

In [ ]:
# The make_multiplier pattern: a classic closure factory
from typing import Callable

def make_multiplier(factor: int) -> Callable[[int], int]:
    """Return a function that multiplies its argument by factor."""
    def multiplier(x: int) -> int:
        return x * factor  # 'factor' is captured from the enclosing scope
    return multiplier

double = make_multiplier(2)
triple = make_multiplier(3)

print(f"double(5) = {double(5)}")   # 10
print(f"triple(5) = {triple(5)}")   # 15
print(f"double(7) = {double(7)}")   # 14

# Inspect the closure's captured variables
print(f"\ndouble's closure vars: {double.__closure__}")
print(f"Captured factor value: {double.__closure__[0].cell_contents}")

In [ ]:
# Practical closure: a counter that maintains state
from typing import Callable

def make_counter(start: int = 0) -> Callable[[], int]:
    """Create a counter function that increments each time it's called."""
    count = start

    def counter() -> int:
        nonlocal count  # Required to modify the enclosed variable
        count += 1
        return count

    return counter

counter_a = make_counter()
counter_b = make_counter(100)

print(f"counter_a: {counter_a()}, {counter_a()}, {counter_a()}")
print(f"counter_b: {counter_b()}, {counter_b()}, {counter_b()}")
# Each counter has its own independent state

In [ ]:
# The nonlocal keyword: modifying enclosed variables
# Without nonlocal, assignment creates a new local variable

def make_accumulator(initial: float = 0.0) -> Callable[[float], float]:
    """Create a function that accumulates a running total."""
    total = initial

    def accumulate(amount: float) -> float:
        nonlocal total  # Without this, 'total += amount' would fail
        total += amount
        return total

    return accumulate

acc = make_accumulator()
print(f"Add 10: {acc(10)}")
print(f"Add 20: {acc(20)}")
print(f"Add 5:  {acc(5)}")

# Common gotcha: the loop closure problem
print("\n--- Loop closure gotcha ---")
functions = []
for i in range(3):
    functions.append(lambda: i)  # All capture the same variable 'i'

# All print 2 because they share the same 'i', which ends at 2
print(f"Gotcha: {[f() for f in functions]}")

# Fix: use a default argument to capture the current value
functions_fixed = []
for i in range(3):
    functions_fixed.append(lambda i=i: i)  # Default captures current value

print(f"Fixed:  {[f() for f in functions_fixed]}")

## Section 4: Lambda Expressions

Lambda expressions create small anonymous functions. They are limited to a single expression
and are best used for short, throwaway functions.

In [ ]:
# Lambda syntax: lambda arguments: expression
square = lambda x: x ** 2
add = lambda a, b: a + b

print(f"square(5) = {square(5)}")
print(f"add(3, 4) = {add(3, 4)}")

# Lambdas are most useful inline as key functions
students = [
    {"name": "Alice", "grade": 92},
    {"name": "Bob", "grade": 85},
    {"name": "Carol", "grade": 98},
]

# Sort by grade using lambda
by_grade = sorted(students, key=lambda s: s["grade"])
print(f"\nBy grade (ascending): {[s['name'] for s in by_grade]}")

by_grade_desc = sorted(students, key=lambda s: s["grade"], reverse=True)
print(f"By grade (descending): {[s['name'] for s in by_grade_desc]}")

# Lambda limitations: only single expressions, no statements
# These are NOT valid lambdas:
# lambda x: if x > 0: return x  (no if statements)
# lambda x: x = 5               (no assignments)

# Conditional expressions ARE allowed (ternary operator)
classify = lambda x: "positive" if x > 0 else "non-positive"
print(f"\nclassify(5) = {classify(5)}")
print(f"classify(-3) = {classify(-3)}")

## Section 5: Practical Closure Examples

Closures are used extensively in real-world Python: callbacks, configuration, logging, and more.

In [ ]:
# Practical example: configurable validator
from typing import Callable

def make_validator(min_val: float, max_val: float) -> Callable[[float], bool]:
    """Create a range validator function."""
    def validate(value: float) -> bool:
        return min_val <= value <= max_val
    return validate

is_valid_percentage = make_validator(0, 100)
is_valid_temperature = make_validator(-273.15, 1000)

print(f"Is 85 a valid percentage? {is_valid_percentage(85)}")
print(f"Is 150 a valid percentage? {is_valid_percentage(150)}")
print(f"Is -300 a valid temperature? {is_valid_temperature(-300)}")
print(f"Is 25 a valid temperature? {is_valid_temperature(25)}")

In [ ]:
# Practical example: callback registry and logger factory
from typing import Callable
from datetime import datetime

def make_logger(prefix: str) -> Callable[[str], None]:
    """Create a logger function with a fixed prefix."""
    def log(message: str) -> None:
        timestamp = datetime.now().strftime("%H:%M:%S")
        print(f"[{timestamp}] {prefix}: {message}")
    return log

debug = make_logger("DEBUG")
error = make_logger("ERROR")

debug("Application started")
error("Connection failed")
debug("Retrying...")

# Practical example: event callback system
print("\n--- Event System ---")

def create_event_system() -> tuple[Callable, Callable, Callable]:
    """Create a simple event system using closures."""
    listeners: list[Callable] = []

    def on(callback: Callable) -> None:
        listeners.append(callback)

    def emit(data: str) -> None:
        for listener in listeners:
            listener(data)

    def count() -> int:
        return len(listeners)

    return on, emit, count

on_message, emit_message, listener_count = create_event_system()

on_message(lambda msg: print(f"  Handler 1: {msg}"))
on_message(lambda msg: print(f"  Handler 2: {msg.upper()}"))

print(f"Registered {listener_count()} listeners")
emit_message("hello world")

In [ ]:
# Closures vs classes: both can encapsulate state
# Closures are lighter-weight for simple stateful functions

# Class approach
class Counter:
    def __init__(self, start: int = 0) -> None:
        self.count = start

    def __call__(self) -> int:
        self.count += 1
        return self.count

# Closure approach (from earlier)
def make_counter(start: int = 0):
    count = start
    def counter() -> int:
        nonlocal count
        count += 1
        return count
    return counter

# Both produce the same behavior
class_counter = Counter()
closure_counter = make_counter()

print("Class counter: ", [class_counter() for _ in range(5)])
print("Closure counter:", [closure_counter() for _ in range(5)])

# When to use which?
# - Closures: simple state, one or two functions, lightweight
# - Classes: complex state, multiple methods, needs inheritance

## Summary

### First-Class Functions
- Functions are objects: assign to variables, store in collections, pass as arguments
- Use `Callable[[ArgTypes], ReturnType]` for type hints on function parameters

### Higher-Order Functions
- Functions that accept functions (`apply_twice`, `sorted` with `key`)
- Functions that return functions (`make_adder`, `make_multiplier`)

### Closures
- Inner functions capture variables from enclosing scope
- Use `nonlocal` to modify enclosed variables
- Watch out for the loop closure gotcha (fix with default arguments)
- Inspect with `func.__closure__`

### Lambda Expressions
- Syntax: `lambda args: expression`
- Limited to single expressions (no statements)
- Best for short, inline key functions

### Practical Patterns
- Factory functions: `make_multiplier`, `make_validator`
- Stateful closures: counters, accumulators
- Callbacks and event systems
- Choose closures for simple state, classes for complex state